
# Criador Interativo de Plano de Corrida (Jack Daniels)

Notebook pensado para mapear o perfil do atleta de forma profunda, gerar zonas e simular um plano com o código do repositório. Use como vitrine do sistema ou como guia para preencher um novo perfil.



## Como funciona
1. Garante que o diretório atual aponta para a raiz do repositório (ou para a pasta clonada).
2. Importa os módulos centrais (`user_profile`, `training_zones`, `plan_generator`).
3. Coleta informações-chave inspiradas no checklist Jack Daniels (objetivo, fisiologia, histórico, disponibilidade, logística, preferências, lesões).
4. Calcula VDOT/zones e gera um plano-simulação com `PlanGenerator` para validar se os dados fazem sentido.
5. Exporta o perfil e o plano em JSON para reaproveitar depois.


## Configuração do Ambiente (Google Colab)
Se você está rodando este notebook no **Google Colab**, execute a célula abaixo para clonar o repositório do GitHub e configurar o ambiente. Se estiver rodando localmente com os arquivos já disponíveis, pode pular esta etapa.

In [ ]:
# Conexão com GitHub (para Google Colab)
# Execute esta célula primeiro se estiver rodando no Colab

import os
import subprocess

# Detecta se está no Google Colab
IN_COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False

if IN_COLAB:
    REPO_URL = 'https://github.com/tallesmedeiros/DecisionMaking.git'
    REPO_DIR = '/content/DecisionMaking'

    if not os.path.exists(REPO_DIR):
        print('Clonando repositório do GitHub...')
        subprocess.run(['git', 'clone', REPO_URL, REPO_DIR], check=True)
        print(f'Repositório clonado em {REPO_DIR}')
    else:
        print(f'Repositório já existe em {REPO_DIR}')
        # Opcional: atualizar o repositório
        print('Atualizando repositório...')
        subprocess.run(['git', '-C', REPO_DIR, 'pull'], check=True)

    os.chdir(REPO_DIR)
    print(f'Diretório atual: {os.getcwd()}')
else:
    print('Não está no Google Colab - pulando clone do GitHub.')

Clonando repositório do GitHub...
Repositório clonado em /content/DecisionMaking
Diretório atual: /content/DecisionMaking


In [ ]:

from pathlib import Path
import os
import sys

REPO_NAME = 'DecisionMaking'
REPO_MARKERS = ('plan_generator.py', 'user_profile.py', 'training_zones.py')

def find_repo_root(start: Path) -> Path:
    for path in [start] + list(start.parents):
        if all((path / marker).exists() for marker in REPO_MARKERS):
            return path
        if (path / REPO_NAME).exists():
            nested = path / REPO_NAME
            if all((nested / marker).exists() for marker in REPO_MARKERS):
                return nested
    return start

starting_dir = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
repo_root = find_repo_root(starting_dir)

if repo_root == starting_dir and not all((repo_root / marker).exists() for marker in REPO_MARKERS):
    raise FileNotFoundError('Não foi possível localizar o repositório DecisionMaking. Clone ou monte o diretório antes de rodar este notebook.')

os.chdir(repo_root)
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print(f'📁 Diretório de trabalho fixado em: {repo_root}')
print('📄 Principais módulos disponíveis:')
for name in sorted(Path(repo_root).glob('*.py')):
    print(f' - {name.name}')


📁 Diretório de trabalho fixado em: /content/DecisionMaking
📄 Principais módulos disponíveis:
 - cli.py
 - environment_analysis.py
 - example_with_zones.py
 - intervals_integration.py
 - notebook_widgets.py
 - pdf_export.py
 - plan_generator.py
 - plot_utils.py
 - running_plan.py
 - test_enhanced.py
 - test_environment_analysis.py
 - test_environment_strategy.py
 - test_environmental_preparation.py
 - test_event_context.py
 - test_example.py
 - test_intervals_upload.py
 - test_new_features.py
 - test_profile_integration.py
 - test_schedule_preferences.py
 - test_visual.py
 - training_zones.py
 - user_profile.py


In [ ]:

from datetime import date
import json
from typing import List, Dict

import ipywidgets as widgets
from IPython.display import display, clear_output

from user_profile import UserProfile, RaceGoal
from training_zones import TrainingZones, RaceTime
from plan_generator import PlanGenerator

print('Bibliotecas carregadas com sucesso e diretório conectado ao repositório.')


Bibliotecas carregadas com sucesso e diretório conectado ao repositório.


## Questionário guiado
Preencha as abas abaixo. Campos essenciais estão marcados, mas sinta-se livre para detalhar ao máximo para gerar planos mais fiéis.


In [ ]:
# Utilidades e schema JSON alvo
from typing import TypedDict, Any

# Schema em dicionário simples para orientar a estrutura de salvamento
PLAN_REQUEST_SCHEMA = {
    "type": "object",
    "required": ["athlete", "goal", "plan_parameters"],
    "properties": {
        "schema_version": {"type": "string"},
        "athlete": {
            "type": "object",
            "required": ["name"],
            "properties": {
                "name": {"type": "string"},
                "age": {"type": "number"},
                "weight_kg": {"type": "number"},
                "height_cm": {"type": "number"},
                "gender": {"type": "string"},
            },
        },
        "goal": {
            "type": "object",
            "required": ["distance", "date"],
            "properties": {
                "distance": {"type": "string"},
                "date": {"type": "string", "format": "date"},
                "name": {"type": "string"},
                "location": {"type": "string"},
                "target_time": {"type": ["string", "null"]},
                "personal_best": {"type": ["string", "null"]},
                "secondary_objectives": {"type": "array", "items": {"type": "string"}},
                "motivation": {"type": "string"},
                "logistics": {"type": "string"},
                "test_races": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "distance": {"type": "string"},
                            "date": {"type": "string", "format": "date"},
                            "name": {"type": "string"},
                            "location": {"type": "string"},
                            "target_time": {"type": ["string", "null"]},
                        },
                    },
                },
            },
        },
        "history": {
            "type": "object",
            "properties": {
                "years_running": {"type": "number"},
                "current_weekly_km": {"type": "number"},
                "average_weekly_km": {"type": "number"},
                "recent_peak_weekly_km": {"type": "number"},
                "initial_weekly_km": {"type": ["number", "null"]},
                "consistent_days_per_week": {"type": "integer"},
                "tolerated_workouts": {"type": "array", "items": {"type": "string"}},
                "adherence_score": {"type": "number"},
                "experience_level": {"type": "string"},
            },
        },
        "availability": {
            "type": "object",
            "properties": {
                "days_per_week": {"type": "integer"},
                "hours_per_day": {"type": "number"},
                "preferred_time": {"type": "string"},
                "preferred_location": {"type": "array", "items": {"type": "string"}},
                "preferred_days": {"type": "array", "items": {"type": "string"}},
                "stressful_blocks": {"type": "object"},
                "long_run_preference_days": {"type": "array", "items": {"type": "string"}},
                "use_alternating_weeks": {"type": "boolean"},
                "alternate_stressful_blocks": {"type": "object"},
                "alternate_long_run_days": {"type": "array", "items": {"type": "string"}},
                "weekly_schedule": {"type": "object"},
                "available_equipment": {"type": "array", "items": {"type": "string"}},
                "default_warmup_minutes": {"type": "integer"},
                "default_cooldown_minutes": {"type": "integer"},
                "commute_minutes": {"type": "integer"},
            },
        },
        "health": {
            "type": "object",
            "properties": {
                "previous_injuries": {"type": "array", "items": {"type": "string"}},
                "current_injuries": {"type": "array", "items": {"type": "string"}},
                "injury_triggers": {"type": "array", "items": {"type": "string"}},
                "red_zones": {"type": "array", "items": {"type": "string"}},
                "strength_routines": {"type": "array", "items": {"type": "string"}},
                "impact_limitations": {"type": "array", "items": {"type": "string"}},
            },
        },
        "preferences": {
            "type": "object",
            "properties": {
                "typical_key_workout_rpe": {"type": "integer"},
                "long_session_tolerance": {"type": "string"},
                "variety_preference": {"type": "string"},
                "social_training_options": {"type": "array", "items": {"type": "string"}},
                "routine_vs_fun_balance": {"type": "string"},
                "session_preferences": {"type": "object"},
                "zone_mix_preference": {"type": "object"},
                "environment": {
                    "type": "object",
                    "properties": {
                        "race_weather": {"type": "string"},
                        "race_elevation": {"type": "string"},
                        "training_terrain": {"type": "array", "items": {"type": "string"}},
                    },
                },
                "feedback_required": {"type": "boolean"},
            },
        },
        "plan_parameters": {
            "type": "object",
            "required": ["plan_name", "goal", "level", "days_per_week"],
            "properties": {
                "plan_name": {"type": "string"},
                "goal": {"type": "string"},
                "level": {"type": "string"},
                "days_per_week": {"type": "integer"},
                "weeks": {"type": ["integer", "null"]},
            },
        },
    },
}

def distance_to_km(label: str) -> float:
    mapping = {'5K': 5.0, '10K': 10.0, 'Half Marathon': 21.0975, 'Marathon': 42.195}
    return mapping.get(label, None)

days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
field_style = {'description_width': '220px'}
DEFAULT_WIDTH = '460px'

def stylize(widget, width: str = DEFAULT_WIDTH, height: str | None = None):
    widget.style = field_style
    widget.layout = widget.layout or widgets.Layout()
    widget.layout.width = width
    if height:
        widget.layout.height = height
    return widget

secondary_objectives_options = [
    'Performance',
    'Saúde geral',
    'Redução de peso',
    'Retomada de ritmo',
    'Consistência',
    'Melhorar VDOT',
]
motivation_options = ['', 'Performance', 'Saúde', 'Retomada', 'Diversão']
tolerated_workouts_options = [
    'Tempo run', 'Intervalos', 'Progressivo', 'Fartlek', 'Subidas', 'Longão leve', 'Cross-training'
]
preference_locations = ['track', 'road', 'trail', 'treadmill']
training_terrain_options = preference_locations + ['montanha', 'misto']
equipment_options = [
    'Relógio GPS', 'Pista', 'Esteira', 'Bike', 'Piscina', 'Academia', 'Trilha', 'Rolo', 'Caminhada'
]
injury_options = ['Joelho', 'Tornozelo', 'Quadril', 'Canela', 'Costas', 'Plantar', 'Outros']
social_training_options = ['Grupo', 'Parceiro fixo', 'Online', 'Solo']
weather_options = ['', 'Quente', 'Úmido', 'Frio', 'Ventoso', 'Variável']
elevation_options = ['', 'Plano', 'Ondulado', 'Montanhoso']

# --- Seção 1: Objetivo e contexto ---
objective_widgets = {
    'athlete_name': stylize(widgets.Text(description='Nome do atleta')),
    'name': stylize(widgets.Text(description='Nome da prova', placeholder='Ex.: Maratona de SP')),
    'location': stylize(widgets.Text(description='Cidade/País', placeholder='Local da prova')),
    'distance': stylize(widgets.Dropdown(options=['5K', '10K', 'Half Marathon', 'Marathon'], description='Distância')),
    'date': stylize(widgets.DatePicker(description='Data')),
    'target_time': stylize(widgets.Text(description='Meta (HH:MM:SS)', placeholder='03:45:00')),
    'personal_best': stylize(widgets.Text(description='PB atual')),
    'test_races': stylize(widgets.Textarea(description='Provas teste', placeholder='10K|2024-08-20|Prova A|00:45:00'), height='72px'),
    'secondary_objectives': stylize(widgets.SelectMultiple(options=secondary_objectives_options, description='Objetivos sec.')),
    'motivation': stylize(widgets.Dropdown(options=motivation_options, description='Motivação')),
    'logistics': stylize(widgets.Textarea(description='Restrições', placeholder='Viagens, clima, terreno...'), height='72px'),
}
objective_box = widgets.VBox(list(objective_widgets.values()))

# --- Seção 2: Fisiologia ---
physiology_widgets = {
    'recent_distance': stylize(widgets.Dropdown(options=['5K', '10K', 'Half Marathon', 'Marathon'], description='Prova recente')),
    'recent_time': stylize(widgets.Text(description='Tempo recente', placeholder='00:45:30')),
    'hr_resting': stylize(widgets.IntText(description='FC repouso', value=0)),
    'hr_max': stylize(widgets.IntText(description='FC máx', value=0)),
    'age': stylize(widgets.IntText(description='Idade')),
    'weight_kg': stylize(widgets.FloatText(description='Peso (kg)')),
    'height_cm': stylize(widgets.FloatText(description='Altura (cm)')),
    'sex': stylize(widgets.Dropdown(options=['', 'M', 'F'], description='Sexo')),
}
physiology_box = widgets.VBox(list(physiology_widgets.values()))

# --- Seção 3: Histórico e consistência ---
history_widgets = {
    'years_running': stylize(widgets.FloatText(description='Anos correndo', value=0.0)),
    'current_weekly_km': stylize(widgets.FloatText(description='Volume atual (km/sem)')),
    'average_weekly_km': stylize(widgets.FloatText(description='Média últimas 8-12 sem')),
    'recent_peak_weekly_km': stylize(widgets.FloatText(description='Pico recente (km)')),
    'initial_weekly_km': stylize(widgets.FloatText(description='Volume inicial desejado', value=0.0)),
    'consistent_days_per_week': stylize(widgets.IntSlider(description='Dias mantidos/sem', value=3, min=1, max=7), width='520px'),
    'tolerated_workouts': stylize(widgets.SelectMultiple(options=tolerated_workouts_options, description='Treinos tolerados')),
    'adherence_score': stylize(widgets.FloatSlider(description='Aderência prévia (%)', value=80, min=0, max=100), width='520px'),
    'experience_level': stylize(widgets.Dropdown(options=['beginner', 'intermediate', 'advanced'], description='Nível')),
}
history_box = widgets.VBox(list(history_widgets.values()))

# --- Seção 4: Disponibilidade ---
availability_widgets = {
    'days_per_week': stylize(widgets.IntSlider(description='Dias/semana alvo', value=4, min=1, max=7), width='520px'),
    'hours_per_day': stylize(widgets.FloatSlider(description='Tempo/sessão (h)', value=1.0, min=0.5, max=3.0, step=0.25), width='520px'),
    'preferred_time': stylize(widgets.Dropdown(options=['', 'morning', 'afternoon', 'evening'], description='Horário')),
    'preferred_days': stylize(widgets.SelectMultiple(options=days_of_week, description='Dias preferidos')),
    'preferred_location': stylize(widgets.SelectMultiple(options=preference_locations, description='Locais preferidos')),
    'weekly_schedule': stylize(widgets.Textarea(description='Grade semanal', placeholder='Seg: 06:00-07:00 | max=60 | terreno=pista'), height='80px'), 'stressful_blocks': stylize(widgets.Textarea(description='Blocos críticos', placeholder='Monday: morning Thursday: night'), height='70px'),
    'long_run_preference_days': stylize(widgets.SelectMultiple(options=days_of_week, description='Dias p/ longão')),
    'use_alternating_weeks': stylize(widgets.Checkbox(description='Alternar semanas (A/B)', value=False)),
    'alternate_stressful_blocks': stylize(widgets.Textarea(description='Blocos críticos semana B'), height='70px'),
    'alternate_long_run_days': stylize(widgets.SelectMultiple(options=days_of_week, description='Longão semana B')),
    'logistics': stylize(widgets.Textarea(description='Infra/acessos', placeholder='Pista às terças, academia...'), height='70px'),
    'available_equipment': stylize(widgets.SelectMultiple(options=equipment_options, description='Equipamentos')),
    'default_warmup_minutes': stylize(widgets.IntText(description='Aquecimento (min)', value=10)),
    'default_cooldown_minutes': stylize(widgets.IntText(description='Desaquecimento (min)', value=10)),
    'commute_minutes': stylize(widgets.IntText(description='Deslocamento (min)', value=0)),
}
availability_box = widgets.VBox(list(availability_widgets.values()))

# --- Seção 5: Lesões e prevenção ---
injury_widgets = {
    'previous_injuries': stylize(widgets.SelectMultiple(options=injury_options, description='Lesões prévias')),
    'current_injuries': stylize(widgets.SelectMultiple(options=injury_options, description='Lesões atuais')),
    'injury_triggers': stylize(widgets.SelectMultiple(options=['Impacto', 'Volume alto', 'Velocidade', 'Terreno'], description='Gatilhos')),
    'red_zones': stylize(widgets.SelectMultiple(options=['Sem subidas', 'Sem tiros', 'Limitar longos', 'Evitar impacto'], description='Restrições')),
    'strength_routines': stylize(widgets.SelectMultiple(options=['Mobilidade', 'Core', 'Treino funcional', 'Levantamento'], description='Rotina de força')),
    'impact_limitations': stylize(widgets.SelectMultiple(options=['Baixo impacto', 'Médio impacto', 'Sem restrição'], description='Limites de impacto')),
}
injury_box = widgets.VBox(list(injury_widgets.values()))

# --- Seção 6: Preferências e psicologia ---
psych_widgets = {
    'rpe_tolerance': stylize(widgets.IntSlider(description='RPE típico (1-10)', value=6, min=1, max=10), width='520px'),
    'long_session_tolerance': stylize(widgets.Dropdown(options=['baixa', 'moderada', 'alta'], description='Tolerância longos')),
    'variety_preference': stylize(widgets.Dropdown(options=['prefere repetição', 'equilíbrio', 'prefere variedade'], description='Variedade')),
    'social_training': stylize(widgets.SelectMultiple(options=social_training_options, description='Treino social')),
    'routine_vs_fun_balance': stylize(widgets.Dropdown(options=['rotina', 'equilibrado', 'diversão'], description='Estilo')),
    'feedback_required': stylize(widgets.Checkbox(description='Solicitar feedback frequente', value=True)),
}
psych_box = widgets.VBox(list(psych_widgets.values()))

# --- Seção 7: Sessões e zonas ---
session_widgets = {
    'intervals': stylize(widgets.Checkbox(description='Intervalos', value=True)),
    'tempo': stylize(widgets.Checkbox(description='Tempo run', value=True)),
    'long_run': stylize(widgets.Checkbox(description='Longão', value=True)),
    'cross_training': stylize(widgets.Checkbox(description='Cross-training', value=False)),
}
zone_mix_widgets = {
    'easy': stylize(widgets.FloatSlider(description='Easy (%)', value=55, min=0, max=100), width='520px'),
    'tempo': stylize(widgets.FloatSlider(description='Tempo (%)', value=25, min=0, max=100), width='520px'),
    'interval': stylize(widgets.FloatSlider(description='Interval (%)', value=20, min=0, max=100), width='520px'),
}
session_box = widgets.VBox(list(session_widgets.values()) + list(zone_mix_widgets.values()))

# --- Seção 8: Clima e terreno ---
climate_widgets = {
    'race_weather': stylize(widgets.Dropdown(options=weather_options, description='Clima da prova')),
    'race_elevation': stylize(widgets.Dropdown(options=elevation_options, description='Altimetria prova')),
    'training_terrain': stylize(widgets.SelectMultiple(options=training_terrain_options, description='Terreno de treino')),
}
climate_box = widgets.VBox(list(climate_widgets.values()))

# Acordeão com todas as seções
accordion = widgets.Accordion(children=[
    objective_box,
    physiology_box,
    history_box,
    availability_box,
    injury_box,
    psych_box,
    session_box,
    climate_box,
])
for i, title in enumerate([
    '1) Objetivo e contexto',
    '2) Fisiologia e VDOT',
    '3) Histórico e consistência',
    '4) Disponibilidade e logística',
    '5) Lesões e prevenção',
    '6) Preferências e psicologia',
    '7) Sessões e mistura de zonas',
    '8) Clima/terreno da prova',
]):
    accordion.set_title(i, title)

display(accordion)



Accordion(children=(VBox(children=(Text(value='', description='Nome do atleta', layout=Layout(width='460px'), …


## Gerar perfil + plano de exemplo
Clique no botão para consolidar as respostas, estimar VDOT com Jack Daniels (se houver prova recente) e gerar um plano exemplo com `PlanGenerator`. Nada é enviado para APIs externas; tudo roda localmente para simular a experiência completa.


In [ ]:
output = widgets.Output()
save_button = widgets.Button(description='Salvar JSON e gerar plano', button_style='success', icon='check')

# Objetos armazenados para reuso fora do callback
generated_profile = None
generated_plan = None

def parse_multiline(value) -> List[str]:
    if isinstance(value, (list, tuple, set)):
        return [str(item).strip() for item in value if str(item).strip()]
    if not value:
        return []
    return [item.strip() for item in str(value).split('') if item.strip()]

def parse_race_list(text: str) -> List[RaceGoal]:
    races = []
    for line in parse_multiline(text):
        parts = [p.strip() for p in line.split('|')]
        if len(parts) >= 2:
            distance, date_str = parts[0], parts[1]
            name = parts[2] if len(parts) > 2 else ''
            target = parts[3] if len(parts) > 3 else None
            location = parts[4] if len(parts) > 4 else ''
            try:
                race_date = date.fromisoformat(date_str)
            except ValueError:
                continue
            races.append(RaceGoal(distance=distance, date=race_date, name=name, location=location, target_time=target, is_main_goal=False))
    return races

def parse_schedule_text(text: str) -> Dict[str, List[Dict[str, object]]]:
    schedule = {}
    for line in parse_multiline(text):
        if ':' not in line:
            continue
        day_part, info = line.split(':', 1)
        day = day_part.strip().capitalize()
        block = {}
        if '|' in info:
            time_part, *extras = [p.strip() for p in info.split('|')]
        else:
            time_part, extras = info.strip(), []
        if '-' in time_part:
            start, end = [t.strip() for t in time_part.split('-', 1)]
            block['start'] = start
            block['end'] = end
        for extra in extras:
            if extra.lower().startswith('max='):
                try:
                    block['max_minutes'] = int(extra.split('=', 1)[1])
                except ValueError:
                    pass
            if 'terreno=' in extra or 'superficie=' in extra or 'superfície=' in extra:
                surfaces = extra.split('=', 1)[1]
                block['surfaces'] = [s.strip() for s in surfaces.split(',') if s.strip()]
        schedule.setdefault(day, []).append(block)
    return schedule

def parse_day_periods(text: str) -> Dict[str, List[str]]:
    mapping = {}
    for line in parse_multiline(text):
        if ':' in line:
            day, periods = line.split(':', 1)
            mapping[day.strip().capitalize()] = [p.strip() for p in periods.split(',') if p.strip()]
        else:
            mapping[line.strip().capitalize()] = []
    return mapping

def normalize_zone_mix(values: Dict[str, float]) -> Dict[str, float]:
    total = sum(values.values()) or 1.0
    return {k: round(v / total, 2) for k, v in values.items()}

def collect_plan_data() -> Dict[str, Any]:
    goal_date = objective_widgets['date'].value or date.today()
    main_goal = {
        'distance': objective_widgets['distance'].value,
        'date': goal_date.isoformat(),
        'name': objective_widgets['name'].value,
        'location': objective_widgets['location'].value,
        'target_time': objective_widgets['target_time'].value or None,
        'personal_best': objective_widgets['personal_best'].value or None,
        'secondary_objectives': list(objective_widgets['secondary_objectives'].value),
        'motivation': objective_widgets['motivation'].value,
        'logistics': objective_widgets['logistics'].value,
        'test_races': [
            {
                'distance': race.distance,
                'date': race.date.isoformat(),
                'name': race.name,
                'location': race.location,
                'target_time': race.target_time,
            }
            for race in parse_race_list(objective_widgets['test_races'].value)
        ],
    }
    schedule = parse_schedule_text(availability_widgets['weekly_schedule'].value)
    zone_mix_raw = {
        'easy': zone_mix_widgets['easy'].value,
        'tempo': zone_mix_widgets['tempo'].value,
        'interval': zone_mix_widgets['interval'].value,
    }
    plan_data = {
        'schema_version': '1.0',
        'athlete': {
            'name': objective_widgets['athlete_name'].value,
            'age': physiology_widgets['age'].value or 0,
            'weight_kg': physiology_widgets['weight_kg'].value or 0.0,
            'height_cm': physiology_widgets['height_cm'].value or 0.0,
            'gender': physiology_widgets['sex'].value,
        },
        'goal': main_goal,
        'history': {
            'years_running': history_widgets['years_running'].value,
            'current_weekly_km': history_widgets['current_weekly_km'].value,
            'average_weekly_km': history_widgets['average_weekly_km'].value,
            'recent_peak_weekly_km': history_widgets['recent_peak_weekly_km'].value,
            'initial_weekly_km': history_widgets['initial_weekly_km'].value or None,
            'consistent_days_per_week': history_widgets['consistent_days_per_week'].value,
            'tolerated_workouts': list(history_widgets['tolerated_workouts'].value),
            'adherence_score': history_widgets['adherence_score'].value,
            'experience_level': history_widgets['experience_level'].value,
        },
        'availability': {
            'days_per_week': availability_widgets['days_per_week'].value,
            'hours_per_day': availability_widgets['hours_per_day'].value,
            'preferred_time': availability_widgets['preferred_time'].value,
            'preferred_location': list(availability_widgets['preferred_location'].value),
            'preferred_days': list(availability_widgets['preferred_days'].value),
            'stressful_blocks': parse_day_periods(availability_widgets['stressful_blocks'].value),
            'long_run_preference_days': list(availability_widgets['long_run_preference_days'].value),
            'use_alternating_weeks': availability_widgets['use_alternating_weeks'].value,
            'alternate_stressful_blocks': parse_day_periods(availability_widgets['alternate_stressful_blocks'].value),
            'alternate_long_run_days': list(availability_widgets['alternate_long_run_days'].value),
            'weekly_schedule': schedule,
            'logistics': availability_widgets['logistics'].value,
            'available_equipment': list(availability_widgets['available_equipment'].value or []),
            'default_warmup_minutes': availability_widgets['default_warmup_minutes'].value,
            'default_cooldown_minutes': availability_widgets['default_cooldown_minutes'].value,
            'commute_minutes': availability_widgets['commute_minutes'].value,
        },
        'health': {
            'previous_injuries': list(injury_widgets['previous_injuries'].value),
            'current_injuries': list(injury_widgets['current_injuries'].value),
            'injury_triggers': list(injury_widgets['injury_triggers'].value),
            'red_zones': list(injury_widgets['red_zones'].value),
            'strength_routines': list(injury_widgets['strength_routines'].value),
            'impact_limitations': list(injury_widgets['impact_limitations'].value),
        },
        'preferences': {
            'typical_key_workout_rpe': psych_widgets['rpe_tolerance'].value,
            'long_session_tolerance': psych_widgets['long_session_tolerance'].value,
            'variety_preference': psych_widgets['variety_preference'].value,
            'social_training_options': list(psych_widgets['social_training'].value),
            'routine_vs_fun_balance': psych_widgets['routine_vs_fun_balance'].value,
            'session_preferences': {k: widget.value for k, widget in session_widgets.items()},
            'zone_mix_preference': normalize_zone_mix(zone_mix_raw),
            'environment': {
                'race_weather': climate_widgets['race_weather'].value,
                'race_elevation': climate_widgets['race_elevation'].value,
                'training_terrain': list(climate_widgets['training_terrain'].value),
            },
            'feedback_required': psych_widgets['feedback_required'].value,
        },
        'plan_parameters': {
            'plan_name': objective_widgets['name'].value or f"Plano {main_goal['distance']}",
            'goal': main_goal['distance'],
            'level': history_widgets['experience_level'].value,
            'days_per_week': availability_widgets['days_per_week'].value,
            'weeks': None,
        },
    }
    recent_time_str = physiology_widgets['recent_time'].value.strip()
    if recent_time_str:
        plan_data['preferences']['recent_race'] = {
            'distance': physiology_widgets['recent_distance'].value,
            'time': recent_time_str,
        }
    return plan_data

def validate_plan_data(plan_data: Dict[str, Any]) -> None:
    missing = []
    if not plan_data['athlete']['name']:
        missing.append('Nome do atleta')
    if not plan_data['goal']['distance']:
        missing.append('Distância alvo')
    if not plan_data['goal']['date']:
        missing.append('Data da prova')
    if not plan_data['plan_parameters']['level']:
        missing.append('Nível de experiência')
    if not plan_data['plan_parameters']['days_per_week']:
        missing.append('Dias/semana')
    if missing:
        raise ValueError('Preencha os campos obrigatórios: ' + ', '.join(missing))

def build_profile(_: widgets.Button):
    global generated_profile, generated_plan
    with output:
        clear_output()
        plan_data = collect_plan_data()
        try:
            validate_plan_data(plan_data)
        except ValueError as err:
            print('⚠️ Validação falhou:', err)
            return

        # Persistência do JSON alinhado ao schema
        json_path = Path('plan_request.json')
        json_path.write_text(json.dumps(plan_data, indent=2, ensure_ascii=False))
        print(f'💾 Dados salvos em {json_path.resolve()}')

        # Converter dados para objetos usados na geração do plano
        main_goal_data = plan_data['goal']
        main_goal = RaceGoal(
            distance=main_goal_data['distance'],
            date=date.fromisoformat(main_goal_data['date']),
            name=main_goal_data['name'],
            location=main_goal_data['location'],
            is_main_goal=True,
            target_time=main_goal_data['target_time'],
        )
        schedule = plan_data['availability']['weekly_schedule']
        zone_mix_preference = plan_data['preferences']['zone_mix_preference']

        profile = UserProfile(
            name=plan_data['athlete']['name'],
            age=plan_data['athlete']['age'],
            weight_kg=plan_data['athlete']['weight_kg'],
            height_cm=plan_data['athlete']['height_cm'],
            gender=plan_data['athlete']['gender'],
            years_running=plan_data['history']['years_running'],
            current_weekly_km=plan_data['history']['current_weekly_km'],
            average_weekly_km=plan_data['history']['average_weekly_km'],
            recent_peak_weekly_km=plan_data['history']['recent_peak_weekly_km'],
            initial_weekly_km=plan_data['history']['initial_weekly_km'],
            consistent_days_per_week=plan_data['history']['consistent_days_per_week'],
            tolerated_workouts=plan_data['history']['tolerated_workouts'],
            adherence_score=plan_data['history']['adherence_score'],
            experience_level=plan_data['history']['experience_level'],
            main_race=main_goal,
            test_races=[RaceGoal.from_dict(race) for race in plan_data['goal'].get('test_races', [])],
            secondary_objectives=plan_data['goal']['secondary_objectives'],
            days_per_week=plan_data['availability']['days_per_week'],
            hours_per_day=plan_data['availability']['hours_per_day'],
            preferred_time=plan_data['availability']['preferred_time'],
            preferred_location=plan_data['availability']['preferred_location'],
            preferred_days=plan_data['availability']['preferred_days'],
            stressful_blocks=plan_data['availability']['stressful_blocks'],
            long_run_preference_days=plan_data['availability']['long_run_preference_days'],
            use_alternating_weeks=plan_data['availability']['use_alternating_weeks'],
            alternate_stressful_blocks=plan_data['availability']['alternate_stressful_blocks'],
            alternate_long_run_days=plan_data['availability']['alternate_long_run_days'],
            weekly_schedule=schedule,
            default_warmup_minutes=plan_data['availability']['default_warmup_minutes'],
            default_cooldown_minutes=plan_data['availability']['default_cooldown_minutes'],
            commute_minutes=plan_data['availability']['commute_minutes'],
            typical_key_workout_rpe=plan_data['preferences']['typical_key_workout_rpe'],
            long_session_tolerance=plan_data['preferences']['long_session_tolerance'],
            variety_preference=plan_data['preferences']['variety_preference'],
            social_training_options=plan_data['preferences']['social_training_options'],
            routine_vs_fun_balance=plan_data['preferences']['routine_vs_fun_balance'],
            recent_race_times={},
            zones_calculation_method='jack_daniels',
            zone_mix_preference=zone_mix_preference,
            hr_resting=physiology_widgets['hr_resting'].value or None,
            hr_max=physiology_widgets['hr_max'].value or None,
            session_preferences=plan_data['preferences']['session_preferences'],
            previous_injuries=plan_data['health']['previous_injuries'],
            current_injuries=plan_data['health']['current_injuries'],
            injury_triggers=plan_data['health']['injury_triggers'],
            red_zones=plan_data['health']['red_zones'],
            available_equipment=plan_data['availability']['available_equipment'],
            strength_routines=plan_data['health']['strength_routines'],
            impact_limitations=plan_data['health']['impact_limitations'],
            feedback_required=plan_data['preferences']['feedback_required'],
        )

        # Race times and VDOT
        zones = None
        recent_race = plan_data['preferences'].get('recent_race')
        if recent_race:
            km = distance_to_km(recent_race['distance'])
            if km:
                try:
                    zones = TrainingZones(method='jack_daniels')
                    zones.add_race_time('recent', RaceTime.from_time_string(km, recent_race['time']))
                    zones.calculate_zones()
                    profile.recent_race_times[recent_race['distance']] = recent_race['time']
                    profile.vdot_estimate = zones.vdot
                except Exception as exc:
                    print('Não foi possível calcular VDOT:', exc)

        # Plano exemplo
        plan = PlanGenerator.generate_plan(
            name=plan_data['plan_parameters']['plan_name'],
            goal=plan_data['plan_parameters']['goal'],
            level=plan_data['plan_parameters']['level'],
            days_per_week=plan_data['plan_parameters']['days_per_week'],
            training_zones=zones,
            user_profile=profile,
        )

        generated_profile = profile
        generated_plan = plan

        # Saídas
        print('📊 Perfil consolidado (conforme schema):')
        print(json.dumps(plan_data, indent=2, ensure_ascii=False))
        print()  # linha em branco

        print('🏃 Plano (resumo JSON):')
        plan_dict = plan.to_dict()
        print(json.dumps(plan_dict, indent=2))
        plan_path = Path('plan_gerado.json')
        plan_path.write_text(json.dumps(plan_dict, indent=2, ensure_ascii=False))
        print(f'💾 Plano salvo em {plan_path.resolve()}')
        print()  # linha em branco
        print('💾 Variáveis geradas: use generated_profile / generated_plan para salvar ou reusar no notebook.')

save_button.on_click(build_profile)
print('Preencha os campos e clique em "Salvar JSON e gerar plano". O resultado aparece abaixo.')
display(save_button, output)



Preencha os campos e clique em "Salvar JSON e gerar plano". O resultado aparece abaixo.


Button(button_style='success', description='Salvar JSON e gerar plano', icon='check', style=ButtonStyle())

Output()

## 🎨 Visualização do Plano de Treinamento

Execute a célula abaixo para visualizar seu plano de treinamento de forma **interativa e colorida**!

O plano será exibido com:
- 🏃 **Emojis** para cada tipo de treino
- 🎨 **Cores** diferenciadas por intensidade
- 📊 **Estatísticas** resumidas
- 💡 **Dicas** de treinamento

In [ ]:
from IPython.display import display, HTML
import json

# 🎨 Configuração visual - Emojis e cores para cada tipo de treino
WORKOUT_CONFIG = {
    'easy': {'emoji': '🚶', 'color': '#90EE90', 'name': 'Corrida Leve', 'icon': '🌿'},
    'recovery': {'emoji': '🧘', 'color': '#98FB98', 'name': 'Recuperação', 'icon': '💆'},
    'long': {'emoji': '🏃‍♂️', 'color': '#87CEEB', 'name': 'Longão', 'icon': '🛤️'},
    'long_run': {'emoji': '🏃‍♂️', 'color': '#87CEEB', 'name': 'Longão', 'icon': '🛤️'},
    'tempo': {'emoji': '⚡', 'color': '#FFD700', 'name': 'Tempo Run', 'icon': '🔥'},
    'threshold': {'emoji': '🎯', 'color': '#FFA500', 'name': 'Limiar', 'icon': '📈'},
    'interval': {'emoji': '💨', 'color': '#FF6B6B', 'name': 'Intervalado', 'icon': '🔄'},
    'intervals': {'emoji': '💨', 'color': '#FF6B6B', 'name': 'Intervalado', 'icon': '🔄'},
    'repetition': {'emoji': '🚀', 'color': '#FF69B4', 'name': 'Repetições', 'icon': '⚡'},
    'rest': {'emoji': '😴', 'color': '#D3D3D3', 'name': 'Descanso', 'icon': '🛋️'},
    'cross': {'emoji': '🚴', 'color': '#DDA0DD', 'name': 'Cross-training', 'icon': '🏊'},
    'cross_training': {'emoji': '🚴', 'color': '#DDA0DD', 'name': 'Cross-training', 'icon': '🏊'},
    'race': {'emoji': '🏆', 'color': '#9370DB', 'name': 'Prova', 'icon': '🎖️'},
    'test_race': {'emoji': '🏁', 'color': '#BA55D3', 'name': 'Prova Teste', 'icon': '📊'},
}

PHASE_EMOJIS = {
    'base': '🏗️ Base',
    'build': '📈 Desenvolvimento',
    'peak': '🔝 Pico',
    'taper': '🎯 Polimento',
    'recovery': '🔄 Recuperação',
}

DAY_EMOJIS = {
    'Monday': '📅 Seg', 'Tuesday': '📅 Ter', 'Wednesday': '📅 Qua',
    'Thursday': '📅 Qui', 'Friday': '📅 Sex', 'Saturday': '📅 Sáb', 'Sunday': '📅 Dom',
}

def get_workout_config(workout_type):
    if not workout_type:
        return {'emoji': '🏃', 'color': '#F5F5F5', 'name': 'Treino', 'icon': '📋'}
    workout_lower = str(workout_type).lower().replace(' ', '_').replace('-', '_')
    for key, config in WORKOUT_CONFIG.items():
        if key in workout_lower or workout_lower in key:
            return config
    return {'emoji': '🏃', 'color': '#F5F5F5', 'name': workout_type, 'icon': '📋'}

def format_duration(minutes):
    if not minutes: return '-'
    try:
        minutes = float(minutes)
        hours = int(minutes // 60)
        mins = int(minutes % 60)
        return f'⏱️ {hours}h{mins:02d}' if hours > 0 else f'⏱️ {mins}min'
    except:
        return '-'

def get_phase_display(phase):
    if not phase: return ''
    for key, display in PHASE_EMOJIS.items():
        if key in str(phase).lower():
            return display
    return f'📋 {phase}'

def safe_float(val, default=0):
    try:
        return float(val) if val else default
    except:
        return default

def display_training_plan_visual(plan):
    if plan is None:
        display(HTML('''
        <div style="text-align:center;padding:40px;background:#fff3cd;border-radius:10px;margin:20px 0;">
            <span style="font-size:48px;">🏃‍♂️</span>
            <h3 style="color:#856404;">Nenhum plano gerado ainda!</h3>
            <p style="color:#856404;">Preencha o formulário e clique em <b>"Salvar JSON e gerar plano"</b></p>
        </div>'''))
        return

    plan_dict = plan.to_dict() if hasattr(plan, 'to_dict') else plan

    # O plano usa 'schedule' para a lista de semanas, não 'weeks'
    # 'weeks' é apenas o número de semanas (inteiro)
    schedule = plan_dict.get('schedule', [])
    num_weeks = plan_dict.get('weeks', len(schedule))

    # Calcular estatísticas
    total_km = 0
    total_sessions = 0

    for week in schedule:
        workouts = week.get('workouts', [])
        if isinstance(workouts, list):
            for w in workouts:
                if isinstance(w, dict):
                    total_sessions += 1
                    total_km += safe_float(w.get('distance_km', 0))

    avg_weekly = total_km / len(schedule) if schedule else 0

    html = f'''
    <style>
        .tp{{font-family:Arial,sans-serif;max-width:100%;}}
        .tp-hero{{background:linear-gradient(135deg,#667eea,#764ba2);color:white;padding:25px;border-radius:15px;text-align:center;margin-bottom:20px;}}
        .tp-hero h1{{margin:0;font-size:24px;}}
        .tp-hero .sub{{opacity:0.9;margin-top:8px;font-size:14px;}}
        .tp-stats{{display:grid;grid-template-columns:repeat(auto-fit,minmax(120px,1fr));gap:12px;margin:20px 0;}}
        .tp-stat{{background:white;border-radius:12px;padding:15px;text-align:center;box-shadow:0 3px 10px rgba(0,0,0,0.1);}}
        .tp-stat .em{{font-size:28px;}}
        .tp-stat .val{{font-size:20px;font-weight:bold;color:#2d3748;}}
        .tp-stat .lab{{font-size:11px;color:#718096;text-transform:uppercase;}}
        .tp-leg{{background:#f8fafc;border-radius:12px;padding:15px;margin-bottom:20px;}}
        .tp-leg-title{{font-weight:600;margin-bottom:10px;}}
        .tp-leg-grid{{display:flex;flex-wrap:wrap;gap:8px;}}
        .tp-leg-item{{display:flex;align-items:center;gap:6px;background:white;padding:6px 12px;border-radius:15px;font-size:12px;}}
        .tp-leg-color{{width:10px;height:10px;border-radius:50%;}}
        .tp-week{{background:white;border-radius:12px;margin-bottom:15px;overflow:hidden;box-shadow:0 3px 10px rgba(0,0,0,0.08);}}
        .tp-week-head{{background:linear-gradient(90deg,#4a5568,#2d3748);color:white;padding:12px 15px;display:flex;justify-content:space-between;}}
        .tp-week-head .title{{font-weight:600;}}
        .tp-week-head .badge{{background:rgba(255,255,255,0.2);padding:4px 10px;border-radius:10px;font-size:11px;}}
        .tp-week-stats{{background:#f8fafc;padding:10px 15px;font-size:12px;color:#4a5568;border-bottom:1px solid #e2e8f0;}}
        .tp-sess{{padding:12px 15px;border-bottom:1px solid #f0f0f0;display:grid;grid-template-columns:100px 1fr 80px 70px;align-items:center;gap:10px;}}
        .tp-sess:last-child{{border-bottom:none;}}
        .tp-sess:hover{{background:#f8fafc;}}
        .tp-sess .day{{font-weight:600;color:#2d3748;font-size:13px;}}
        .tp-sess .type{{display:inline-flex;align-items:center;gap:6px;padding:5px 12px;border-radius:15px;font-size:12px;font-weight:600;}}
        .tp-sess .desc{{color:#4a5568;font-size:12px;margin-top:4px;}}
        .tp-sess .met{{text-align:right;font-size:12px;color:#718096;}}
        .tp-tips{{background:linear-gradient(135deg,#e0f7fa,#b2ebf2);border-radius:12px;padding:15px;margin-top:20px;}}
        .tp-tips h3{{margin:0 0 12px;color:#00796b;font-size:14px;}}
        .tp-tips .tip{{display:flex;align-items:center;gap:8px;font-size:13px;color:#004d40;margin:6px 0;}}
        @media(max-width:600px){{.tp-sess{{grid-template-columns:1fr;}} .tp-sess .met{{text-align:left;}}}}
    </style>
    <div class="tp">
        <div class="tp-hero">
            <h1>🏃‍♂️ {plan_dict.get('name', 'Plano de Treinamento')}</h1>
            <div class="sub">🎯 {plan_dict.get('goal', '-')} | 📊 {plan_dict.get('level', '-')} | 📅 {plan_dict.get('days_per_week', '-')} dias/sem</div>
        </div>
        <div class="tp-stats">
            <div class="tp-stat"><div class="em">📆</div><div class="val">{num_weeks}</div><div class="lab">Semanas</div></div>
            <div class="tp-stat"><div class="em">🏃</div><div class="val">{total_sessions}</div><div class="lab">Sessões</div></div>
            <div class="tp-stat"><div class="em">📏</div><div class="val">{total_km:.0f}km</div><div class="lab">Total</div></div>
            <div class="tp-stat"><div class="em">📊</div><div class="val">{avg_weekly:.1f}km</div><div class="lab">Média/Sem</div></div>
        </div>
        <div class="tp-leg">
            <div class="tp-leg-title">🎨 Tipos de Treino</div>
            <div class="tp-leg-grid">'''

    seen = set()
    for key, cfg in WORKOUT_CONFIG.items():
        if cfg['name'] not in seen and key not in ['long_run', 'intervals', 'cross_training']:
            seen.add(cfg['name'])
            html += f'<div class="tp-leg-item"><span class="tp-leg-color" style="background:{cfg["color"]}"></span>{cfg["emoji"]} {cfg["name"]}</div>'

    html += '</div></div>'

    for week in schedule:
        week_num = week.get('week_number', '?')
        phase = get_phase_display(week.get('phase', week.get('notes', '')))
        workouts = week.get('workouts', [])
        week_km = safe_float(week.get('total_distance_km', 0))
        if week_km == 0 and isinstance(workouts, list):
            week_km = sum(safe_float(w.get('distance_km', 0)) for w in workouts if isinstance(w, dict))
        num_workouts = len(workouts) if isinstance(workouts, list) else 0

        html += f'''<div class="tp-week">
            <div class="tp-week-head"><span class="title">📅 Semana {week_num}</span><span class="badge">{phase}</span></div>
            <div class="tp-week-stats">📏 {week_km:.1f}km | 🏃 {num_workouts} sessões</div>'''

        if isinstance(workouts, list):
            for workout in workouts:
                if not isinstance(workout, dict):
                    continue
                wtype = workout.get('workout_type', workout.get('type', 'easy'))
                cfg = get_workout_config(wtype)
                day = workout.get('day', workout.get('day_of_week', '-'))
                day_str = DAY_EMOJIS.get(day, f'📅 {day}')
                desc = workout.get('description', workout.get('details', '-'))
                dist = safe_float(workout.get('distance_km', 0))
                dur = safe_float(workout.get('duration_minutes', workout.get('duration', 0)))
                dist_str = f'📏 {dist:.1f}km' if dist else ''
                dur_str = format_duration(dur) if dur else ''

                html += f'''<div class="tp-sess">
                    <div class="day">{day_str}</div>
                    <div><span class="type" style="background:{cfg['color']}">{cfg['emoji']} {cfg['name']}</span><div class="desc">{cfg['icon']} {desc}</div></div>
                    <div class="met">{dist_str}</div>
                    <div class="met">{dur_str}</div>
                </div>'''

        html += '</div>'

    html += '''<div class="tp-tips">
        <h3>💡 Dicas para seu treino</h3>
        <div class="tip">🧊 Aquecimento e desaquecimento em cada sessão</div>
        <div class="tip">💧 Mantenha-se hidratado durante todo o dia</div>
        <div class="tip">😴 Priorize o sono para melhor recuperação</div>
        <div class="tip">📝 Registre como se sentiu após cada treino</div>
        <div class="tip">🎧 Ouça seu corpo - descanse se necessário</div>
    </div></div>'''

    display(HTML(html))

print("=" * 60)
print("🏃‍♂️ VISUALIZAÇÃO DO PLANO DE TREINAMENTO")
print("=" * 60)

if 'generated_plan' in dir() and generated_plan is not None:
    display_training_plan_visual(generated_plan)
else:
    display_training_plan_visual(None)


🏃‍♂️ VISUALIZAÇÃO DO PLANO DE TREINAMENTO
